In [1]:

import pandas as pd
import scipy
import pyodbc

import time

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-IFK6D8L3\\SQLEXPRESS;DATABASE=StockQuote;UID=sa;PWD=password')
cur = conn.cursor()


SQL = """ select Script_Name, Trnx_date trnx, [close]cls from NSE_EOD where Trnx_date >= '2021-01-01' and Script_Name = 'ZYDUSWELL' order by Script_Name,Trnx_date
"""

df = pd.read_sql(SQL,conn)
display(df)

,Script_Name,trnx,cls
0,ZYDUSWELL,2021-01-04,2148.40
1,ZYDUSWELL,2021-01-05,2113.60
2,ZYDUSWELL,2021-01-06,2085.45
3,ZYDUSWELL,2021-01-07,2061.55
4,ZYDUSWELL,2021-01-08,2065.60
...,...,...,...
484,ZYDUSWELL,2022-12-20,1588.85
485,ZYDUSWELL,2022-12-21,1571.95
486,ZYDUSWELL,2022-12-22,1542.50
487,ZYDUSWELL,2022-12-23,1493.55


In [6]:
df_gby = (
        (df
            .groupby(['Script_Name', 'trnx'])['cls']
            .mean()
        )
        .ewm(span=12)
        .mean()
        # .droplevel(2)
        # .reset_index(name='ema'+str(days))
    )

df_gby

Script_Name  trnx      
ZYDUSWELL    2021-01-04    2148.400000
             2021-01-05    2129.550000
             2021-01-06    2112.337760
             2021-01-07    2096.306049
             2021-01-08    2087.963325
                              ...     
             2022-12-20    1598.901145
             2022-12-21    1594.754815
             2022-12-22    1586.715613
             2022-12-23    1572.382442
             2022-12-26    1565.531297
Name: cls, Length: 489, dtype: float64

In [7]:
def ewm(df, days):
    df_gby = (
        (df
            .groupby(['Script_Name', 'trnx'])['cls']
            .sum()
        )
        .ewm(span=days, min_periods=days)
        .mean()
        .droplevel(1)
        .reset_index(name='ema'+str(days))
    )
    
    return df_gby['ema'+str(days)]

In [8]:
for i in [5,9,12,21,25,26,50,100,200]:
    df['ema'+str(i)] = ewm(df, i)

df

,Script_Name,trnx,cls,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD
0,ZYDUSWELL,2021-01-04,2148.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZYDUSWELL,2021-01-05,2113.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZYDUSWELL,2021-01-06,2085.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZYDUSWELL,2021-01-07,2061.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZYDUSWELL,2021-01-08,2065.60,2079.688626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,ZYDUSWELL,2022-12-20,1588.85,1594.858946,1597.727345,1598.901145,1604.996842,1608.531555,1609.414895,1624.372691,1632.680149,1665.327103,-10.513750
485,ZYDUSWELL,2022-12-21,1571.95,1587.222630,1592.571876,1594.754815,1601.992583,1605.717589,1606.639718,1622.316899,1631.477499,1664.390720,-11.884903
486,ZYDUSWELL,2022-12-22,1542.50,1572.315087,1582.557501,1586.715613,1596.584167,1600.854697,1601.888628,1619.186825,1629.715465,1663.168499,-15.173015
487,ZYDUSWELL,2022-12-23,1493.55,1546.060058,1564.756001,1572.382442,1587.217424,1592.600490,1593.863544,1614.259891,1627.018963,1661.467834,-21.481103


In [9]:
df['MACD'] = df['ema12']-df['ema26']
display(df)
#df['Signal'] = ewm(df['MACD'],9)
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

,Script_Name,trnx,cls,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD
0,ZYDUSWELL,2021-01-04,2148.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZYDUSWELL,2021-01-05,2113.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZYDUSWELL,2021-01-06,2085.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZYDUSWELL,2021-01-07,2061.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZYDUSWELL,2021-01-08,2065.60,2079.688626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,ZYDUSWELL,2022-12-20,1588.85,1594.858946,1597.727345,1598.901145,1604.996842,1608.531555,1609.414895,1624.372691,1632.680149,1665.327103,-10.513750
485,ZYDUSWELL,2022-12-21,1571.95,1587.222630,1592.571876,1594.754815,1601.992583,1605.717589,1606.639718,1622.316899,1631.477499,1664.390720,-11.884903
486,ZYDUSWELL,2022-12-22,1542.50,1572.315087,1582.557501,1586.715613,1596.584167,1600.854697,1601.888628,1619.186825,1629.715465,1663.168499,-15.173015
487,ZYDUSWELL,2022-12-23,1493.55,1546.060058,1564.756001,1572.382442,1587.217424,1592.600490,1593.863544,1614.259891,1627.018963,1661.467834,-21.481103


In [10]:
display(df)

,Script_Name,trnx,cls,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD,Signal
0,ZYDUSWELL,2021-01-04,2148.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZYDUSWELL,2021-01-05,2113.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZYDUSWELL,2021-01-06,2085.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZYDUSWELL,2021-01-07,2061.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZYDUSWELL,2021-01-08,2065.60,2079.688626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,ZYDUSWELL,2022-12-20,1588.85,1594.858946,1597.727345,1598.901145,1604.996842,1608.531555,1609.414895,1624.372691,1632.680149,1665.327103,-10.513750,-12.883639
485,ZYDUSWELL,2022-12-21,1571.95,1587.222630,1592.571876,1594.754815,1601.992583,1605.717589,1606.639718,1622.316899,1631.477499,1664.390720,-11.884903,-12.683892
486,ZYDUSWELL,2022-12-22,1542.50,1572.315087,1582.557501,1586.715613,1596.584167,1600.854697,1601.888628,1619.186825,1629.715465,1663.168499,-15.173015,-13.181716
487,ZYDUSWELL,2022-12-23,1493.55,1546.060058,1564.756001,1572.382442,1587.217424,1592.600490,1593.863544,1614.259891,1627.018963,1661.467834,-21.481103,-14.841594


In [23]:
df.to_csv("./test11.csv")